# APPLIED DATA SCIENCE CAPSTONE WEEK 5 

### Please refer to report for further information about the analysis and insights driven from the data 

#### Case : Shopping Center or Mall in Kuala Lampur

In [5]:
# IMPORTING LIBRARIES

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

In [8]:
# DATA SCRAPING AND MANIPULATION

data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [9]:
kl_df.shape

(71, 1)

In [10]:
# GEOGRAPHICAL COORDINATES

def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

coords

[[3.0576900000000364, 101.74388000000005],
 [3.148494115588384, 101.69672876508707],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1292000000000257, 101.67844000000008],
 [3.111020000000053, 101.66283000000004],
 [3.061870000000056, 101.74675000000008],
 [3.147890000000075, 101.69405000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.0578100000000745, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.1511418740671178, 101.69937468268016],
 [3.129290000000026, 101.69896000000006],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.1479700000000435, 101.66795000000008],
 [3.136442497863743, 101.69029641203808],
 [3.1419057131529953, 101.67967833034047],
 [3.1566851613075664, 101.6

In [13]:
# CREATING THE COORDINATES DATAFRAME

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

print(kl_df.shape)
kl_df

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148494,101.696729
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610
5,Bandar Tun Razak,3.082800,101.722810
6,Bangsar,3.129200,101.678440
7,Bangsar Park,3.129200,101.678440
8,Bangsar South,3.111020,101.662830
9,Batu 11 Cheras,3.061870,101.746750


In [14]:
kl_df.to_csv("kl_df.csv", index=False)

In [15]:
# KUALA LAMPUR COORDINATES

address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516964, 101.6942371.


In [16]:
# CREATING A MAP OF KUALA LAMPUR
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [20]:
# SAVE TO HTML file
map_kl.save('map_kl.html')

In [22]:
# FOURSQUARE APPLICATION

CLIENT_ID = 'D5GYQFOYFHEZNSIP5KTR3SDSOSP2MBGKJUGBKFAS0RW1O3AC'
CLIENT_SECRET = 'GICBOT1QIWCJO30SBVY41DNSVRJORCJVCY0Z2K0SL3LKC2CQ'
VERSION = '20180605'

In [ ]:
# RADIUS 2000M / VENUES 100

radius = 2000
LIMIT = 100
venues = []
for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    results = requests.get(url).json()["response"]["groups"][0]["items"]
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [26]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2599, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
2,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
3,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant
4,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant


In [27]:
#Count the VANUES

venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,99,99,99,99,99,99
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


In [28]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 240 uniques categories.


In [29]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Supplement Shop', 'Food & Drink Shop', 'Noodle House',
       'Seafood Restaurant', 'Chinese Restaurant', 'Restaurant',
       'Breakfast Spot', 'Vegetarian / Vegan Restaurant',
       'Dim Sum Restaurant', 'Food Court', 'Convenience Store', 'Park',
       'Bubble Tea Shop', 'Snack Place', 'Indian Restaurant',
       'Dessert Shop', 'Spa', 'Asian Restaurant', 'Other Great Outdoors',
       'Café', 'Coffee Shop', 'Outlet Store', 'Chinese Breakfast Place',
       'Bakery', 'Pet Store', 'Farmers Market', 'Food Truck',
       'Mamak Restaurant', 'Gym / Fitness Center', 'Hakka Restaurant',
       'Japanese Restaurant', 'Malay Restaurant', 'Athletics & Sports',
       'Fast Food Restaurant', 'Cantonese Restaurant', 'Badminton Court',
       'Burger Joint', 'Middle Eastern Restaurant', 'Steakhouse',
       'College Bookstore', 'Grocery Store', 'Halal Restaurant', 'Winery',
       'Basketball Court', 'Salon / Barbershop', 'Hostel', 'Exhibit',
       'Chettinad Restaurant', 'Hotel', 'So

In [30]:
#ANALYSING NEIGHBORHOODS

# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2599, 241)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Betting Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Donut Shop,Dumpling Restaurant,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Field,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Hill,Historic Site,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Winery,Women's Store,Yoga Studio,Zoo
0,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

In [31]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(26, 241)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Betting Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Donut Shop,Dumpling Restaurant,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Field,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Hill,Historic Site,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Winery,Women's Store,Yoga Studio,Zoo
0,Alam Damai,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.07000,0.01,0.00,0.00,0.00,0.00,0.010000,0.010000,0.000000,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.020000,0.00,0.00,0.010000,0.00,0.010000,0.00,0.00,0.00,0.030000,0.00,0.02,0.000000,0.00,0.01,0.190000,0.00,0.00,0.00,0.00,0.010000,0.00,0.01,0.000000,0.00,0.060000,0.00,0.00,0.00,0.00,0.000000,0.010000,0.020000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.01,0.010000,0.00,0.000000,0.00,0.00,0.01,0.010000,0.00,0.030000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.01,0.000000,0.010000,0.000000,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.0

In [33]:
# CREATING NEW DATAFRAME FOR SHOPPING MALLS

kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.000000
1,"Ampang, Kuala Lumpur",0.010000
2,Bandar Menjalara,0.010000
3,Bandar Sri Permaisuri,0.000000
4,Bandar Tasik Selatan,0.020202


In [34]:
# CLUSTERING NEIGHBORHOODS USING K-MEANS

kclusters = 3
kl_clustering = kl_mall.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)
kmeans.labels_[0:10]

array([1, 1, 1, 1, 0, 1, 2, 2, 0, 1], dtype=int32)

In [36]:
kl_merged = kl_mall.copy()
kl_merged["Cluster Labels"] = kmeans.labels_
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.000000,1
1,"Ampang, Kuala Lumpur",0.010000,1
2,Bandar Menjalara,0.010000,1
3,Bandar Sri Permaisuri,0.000000,1
4,Bandar Tasik Selatan,0.020202,0


In [38]:
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(26, 3)


,Neighborhood,Shopping Mall,Cluster Labels
12,Bukit Bintang,0.030000,0
23,Dang Wangi,0.020000,0
22,"Damansara, Kuala Lumpur",0.030000,0
20,Damansara Heights,0.020000,0
19,Chow Kit,0.020000,0
17,Bukit Tunku,0.020000,0
15,Bukit Nanas,0.030000,0
8,Bangsar South,0.020000,0
25,"Federal Hill, Kuala Lumpur",0.020000,0
4,Bandar Tasik Selatan,0.020202,0


In [ ]:
# CREATING MAP CLUSTERS


map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [40]:
# SAVE MAP AS HTML file
map_clusters.save('map_clusters.html')

In [41]:
#CLUSTER 1

kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels
12,Bukit Bintang,0.030000,0
23,Dang Wangi,0.020000,0
22,"Damansara, Kuala Lumpur",0.030000,0
20,Damansara Heights,0.020000,0
19,Chow Kit,0.020000,0
17,Bukit Tunku,0.020000,0
15,Bukit Nanas,0.030000,0
8,Bangsar South,0.020000,0
25,"Federal Hill, Kuala Lumpur",0.020000,0
4,Bandar Tasik Selatan,0.020202,0


In [42]:
# CLUSTER 2

kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.00,1
16,Bukit Petaling,0.01,1
10,"Batu, Kuala Lumpur",0.00,1
24,Desa Petaling,0.00,1
13,Bukit Jalil,0.00,1
14,Bukit Kiara,0.00,1
2,Bandar Menjalara,0.01,1
18,"Cheras, Kuala Lumpur",0.00,1
3,Bandar Sri Permaisuri,0.00,1
21,Damansara Town Centre,0.01,1


In [43]:
# CLUSTER 3

kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels
6,Bangsar,0.05,2
7,Bangsar Park,0.05,2
11,Brickfields,0.04,2


### CONCLUTSION AND INSIGHT

Shopping malls are concentrated in the central area of Kuala Lumpur city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls.